In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import re

In [2]:
# 크롤링 정보 정리 (특수문자 제거 및 줄바꿈 제거)
def remove_special_characters(text):

    # 특수문자 제거
    text_str = str(text)
    cleaned_text = re.sub(r'[^\w가-힣\s.%~]', '', text_str)

    # 여러 개의 공백을 하나의 공백으로 변환
    cleaned_text = ' '.join(cleaned_text.split())

    return cleaned_text

In [3]:
# Premium 카드 혜택 요약 정보 크롤링 함수
def premium_card_summary(driver, card_url):
    driver.get(card_url)
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 카드 요약 정보 딕셔너리
    premium_card_summary_data = {
        "Card Summary": []
    }

    # 카드 요약 크롤링 코드
    card_summary_container = soup.find('div', class_='card_benefit_wrap')
    card_summary_list = card_summary_container.find_all('dt', class_='h1_eb_lt')
    for summary_item in card_summary_list:
        premium_card_summary_data['Card Summary'].append(summary_item.text)
    
    return premium_card_summary_data

In [4]:
# M ~ ZERO 카드 혜택 요약 정보 크롤링 함수
def crawl_card_summary(driver, card_url):
    driver.get(card_url)
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 카드 요약 정보 딕셔너리
    card_summary_data = {
        "Card Summary": []
    }

    # 카드 요약 크롤링 코드
    card_summary_container = soup.find('ul', class_='h4_m')
    card_summary_list = card_summary_container.find_all('li')
    for summary_item in card_summary_list:
        card_summary_data['Card Summary'].append(summary_item.text)

    return card_summary_data

In [5]:
# 카드 혜택 상세 정보 크롤링 함수
def crawl_card_benefit(driver, card_url):
    driver.get(card_url)
    driver.implicitly_wait(10)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 카드 상세 정보 딕셔너리
    card_info = {
        "Card Benefit": []
    }

    popup_containers = soup.find_all('div', class_='modal_pop')
    for popup in popup_containers:
        title = popup.find('div', class_='layer_head')
        if title and (title.text.strip()[-2:] == "적립" or title.text.strip()[-2:] == "사용" 
                      or title.text.strip()[-2:] == "할인" or title.text.strip()[-5:] == "업그레이드"
                      or title.text.strip()[-3:] == "보너스" or title.text.strip()[-3:] == "서비스"
                      or title.text.strip()[-3:] == "트래블" or title.text.strip()[-3:] == "리워드"):
            contents_string = popup.find('div', class_='box_content')
            contents = contents_string.find_all('div', class_='big_accod')
            content_title = contents_string.find('div', class_='box_tit')

            for index, content in enumerate(contents):
                if content_title and content_title.text == "특수 소재 플레이트":
                    if index == 0:
                        continue
                benefits = remove_special_characters(content.text)
                card_info['Card Benefit'].append(benefits)
                
            z_contents = contents_string.find_all('div', class_='box_line')
            for z_content in z_contents:
                z_benefits = remove_special_characters(z_content.text)
                card_info['Card Benefit'].append(z_benefits)

    return card_info

In [6]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 카드 리스트 페이지 URL
card_list_url = "https://www.hyundaicard.com/cpc/ma/CPCMA0101_01.hc?cardflag=ALL"
driver.get(card_list_url)
driver.implicitly_wait(10)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 카드 리스트 및 혜택 정보 담는 객체
card_list = []

# 카드 리스트 링크 가져오기
card_sections = soup.find('div', class_='card_section')
card_section = card_sections.find_all('div', class_='card_section')

for index in range(0, 5):
    card_data = card_section[index]
    cards = card_data.find_all('div', class_='card_plt')
    for card in cards:
        # 카드 이름 정보
        card_name = card.find('span', class_='h4_b_lt')
        print(card_name.text)

        # 카드 이미지 정보
        card_img = card.find('img')
        card_img_url = card_img.get('src')
        print(card_img_url)

        # 카드 고유 번호 추출
        card_url = card.find('a')
        card_url_code = card_url.get('onclick')
        start_index = card_url_code.find("'") + 1
        end_index = card_url_code.find("'", start_index)
        card_code = card_url_code[start_index:end_index]
        card_url = f"https://www.hyundaicard.com/cpc/cr/CPCCR0201_01.hc?cardWcd={card_code}"

        # 카드 혜택 요약 정보
        # Premium 카드 라인
        if index == 0:
            card_summary_benefit = premium_card_summary(driver, card_url)

        # M ~ ZERO 카드 라인
        else:    
            card_summary_benefit = crawl_card_summary(driver, card_url)
            
        clean_card_summary_benefit = remove_special_characters(card_summary_benefit)
        print(clean_card_summary_benefit)

        # 카드 혜택 상세 정보
        card_detail_benefit = crawl_card_benefit(driver, card_url)
        clean_card_detail_benefit = remove_special_characters(card_detail_benefit)
        print(clean_card_detail_benefit)

driver.quit()

the Black Edition3
https://img.hyundaicard.com/img/com/card/card_TBE3_h.png
Card Summary Invitation Only 1st 1000
Card Benefit 특수 소재 플레이트 여행 항공권 무료 업그레이드동반자 할인 대한항공형 대한항공 퍼스트 클래스 업그레이드 가능 아시아나항공형M포인트형 아시아나항공 비즈니스 스위트 클래스 무료 업그레이드와 마일리지 추가 적립 또는 비즈니즈 클래스 동반 1인 항공권 50% 할인택 1 인천국제공항 제2터미널 내 마티나 골드 라운지 무료 이용 이용 항공사 및 좌석 등급 무관 코로나19로 인한 휴업 기간 중 마티나 라운지 무료 이용 서비스로 대체 PRIVIA 이용 시 국내외 항공권 10% 할인 전월 이용금액본인가족법인카드 20만원 이상 시 항공료 구매금액 기준 연간 1천만원까지 할인 혜택 제공 대한항공 아시아나항공 등 일부 항공사 이용 시 발권대행료 국내선 5% 국제선 7% 부과 인천국제공항 무료 발레파킹 전월 이용금액본인가족법인카드 30만원 이상 시 이용 가능신규 회원은 가입월 포함 2개월간 전월 이용금액 무관하게 혜택 제공 발레파킹 외 주차요금 별도 부담 유의사항 자세한 내용은 카드 발급 후 배송되는 디렉토리를 참고하시기 바랍니다. 일부 혜택과 서비스는 제휴사 및 현대카드의 사정에 따라 변경되거나 중단될 수 있습니다. 각 항공사별 발권대행료 등에 대한 자세한 내용은 홈페이지 컬처라이프스타일 PRIVIA 여행을 참고해 주시기 바랍니다. 골프 제휴 골프장 주중 회원대우서비스 주중 경기도 남양주 및 이천 지역 제휴 골프장 회원대우 할인 혜택 제공주말 불가 the Black Edition3로 그린피 결제 시 제휴 골프장별 연간 1회에 한해 혜택 제공 골프장 부킹 대행 및 문자메시지 알림서비스 골프장 회원권 소지 회원 중 해당 골프장 부킹 요청 시 대행 예약일자 2일 전 문자메시지로 일정 알림서비스 제공 유의사항 자세한 내용은 카드 발급 후 배송되는 디렉토리를 참고하시